### Topic Modeling

- All the document in the collection share the same set of topics, but each document exhibits those topics in different proportions

- Each word in each document is drawn from one of the topics, where the selected topic is chosen from the per-document distribution over topic

Install the library, `gensim`

In [2]:
!pip install gensim

  Retrying (Retry(total=4, connect=None, read=None, redirect=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', timeout('timed out',))': /simple/gensim/
    100% |████████████████████████████████| 15.1MB 71kB/s eta 0:00:01
  Running setup.py bdist_wheel for gensim ... - \ | / - \ | / - \ | / - \ done
  Stored in directory: /home/rs/jpatro/.cache/pip/wheels/6f/f0/82/34d132e5a03ca765ec7f006d5f38ec7d749274a3f59c3fa5d0
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /home/rs/jpatro/.cache/pip/wheels/b0/81/ad/856aade935fceaab491a800ec4de58edb8642afa4c4ba91a00
  Running setup.py bdist_wheel for bz2file ... - \ done
  Stored in directory: /home/rs/jpatro/.cache/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built gensim smart-open bz2file


In [3]:
from gensim import corpora

In [4]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

This is a tiny corpus of nine documents, each consisting of only a single sentence.

First, let’s tokenize the documents, remove common words (using a toy stoplist) as well as words that only appear once in the corpus:

In [5]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Find all words which occur more than 1 time in the corpus (of 9 documents)
texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [6]:
dictionary = corpora.Dictionary(texts)
dictionary.save('deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)


In [7]:
print(dictionary.token2id)

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'user': 4, 'system': 5, 'response': 6, 'time': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [8]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (2, 1)]


The function `doc2bow()` simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a sparse vector. The sparse vector `[(word_id, 1), (word_id, 1)]` therefore reads: in the document *“Human computer interaction”*, the words *"computer"* and *"human"*, identified by an integer id given by the built dictionary, appear once; the other ten dictionary words appear (implicitly) zero times. Check their id at the dictionary displayed in the previous cell and see that they match.

In [9]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('deerwester.mm', corpus)  # store to disk, for later use
for c in corpus:
    print(c)

[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


## LDA

In [10]:
from gensim.corpora import Dictionary
from gensim.models import ldamodel
import numpy
%matplotlib inline


In [11]:
texts = [['bank','river','shore','water'],
        ['river','water','flow','fast','tree'],
        ['bank','water','fall','flow'],
        ['bank','bank','water','rain','river'],
        ['river','water','mud','tree'],
        ['money','transaction','bank','finance'],
        ['bank','borrow','money'], 
        ['bank','finance'],
        ['finance','money','sell','bank'],
        ['borrow','sell'],
        ['bank','loan','sell']]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(0, 1), (3, 1), (4, 1), (7, 1)],
 [(0, 2), (1, 1), (3, 1), (8, 1)],
 [(1, 1), (3, 1), (6, 1), (9, 1)],
 [(0, 1), (10, 1), (11, 1), (12, 1)],
 [(0, 1), (10, 1), (13, 1)],
 [(0, 1), (12, 1)],
 [(0, 1), (10, 1), (12, 1), (14, 1)],
 [(13, 1), (14, 1)],
 [(0, 1), (14, 1), (15, 1)]]

In [12]:
numpy.random.seed(1) # setting random seed to get the same results each time.
model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=2)

In [13]:
model.show_topics()

[(0,
  '0.206*"bank" + 0.111*"water" + 0.083*"river" + 0.083*"sell" + 0.065*"flow" + 0.065*"borrow" + 0.064*"money" + 0.051*"finance" + 0.049*"tree" + 0.041*"rain"'),
 (1,
  '0.136*"bank" + 0.103*"finance" + 0.101*"water" + 0.099*"river" + 0.084*"money" + 0.061*"transaction" + 0.061*"tree" + 0.055*"mud" + 0.054*"sell" + 0.043*"shore"')]

In [14]:
model.get_term_topics('water')

[(0, 0.097919175370194289), (1, 0.081618646341786241)]

In [15]:
model.get_term_topics('finance')

[(0, 0.037650404355932685), (1, 0.083387133953968046)]

In [18]:
bow_water = ['bank','water','bank']
bow_finance = ['bank','finance','bank']

bow = model.id2word.doc2bow(bow_water) # convert to bag of words format first
print (bow)
doc_topics, word_topics, phi_values = model.get_document_topics(bow, per_word_topics=True)

print (word_topics)
print (doc_topics)
#print (phi_values)

[(0, 2), (3, 1)]
[(0, [0, 1]), (3, [0, 1])]
[(0, 0.82570570581386704), (1, 0.17429429418613296)]


Now what does that output mean? It means that like `word_type 1`, our `word_type` `3`, which is the word `bank`, is more likely to be in `topic_0` than `topic_1`.

You must have noticed that while we unpacked into `doc_topics` and `word_topics`, there is another variable - `phi_values`. Like the name suggests, phi_values contains the phi values for each topic for that particular word, scaled by feature length. Phi is essentially the probability of that word in that document belonging to a particular topic. The next few lines should illustrate this. 

In [16]:
phi_values

[(0, [(0, 0.92486455564294345), (1, 0.075135444357056574)]),
 (3, [(0, 1.5817120973072454), (1, 0.41828790269275457)])]

This means that `word_type` 0 has the following phi_values for each of the topics. 
What is intresting to note is `word_type` 3 - because it has 2 occurences (i.e, the word `bank` appears twice in the bow), we can see that the scaling by feature length is very evident. The sum of the phi_values is 2, and not 1.